# import library

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install wxee rasterio

In [ ]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd
import wxee
import rasterio
import numpy as np
from datetime import datetime

# import constant

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

# import xarray sample

In [ ]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')
soil_wc

# GEE

In [ ]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/w/private-key.json"
)

ee.Initialize(credentials)

# get lithology data as xarray

In [ ]:
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])

lithology = ee.Image("CSP/ERGo/1_0/US/lithology")


# Crop Mask selection

lithology_c = lithology.select('b1').clip(shape)

# cultivated = CDL.filterDate('2022-01-01','2022-12-31').select('cultivated').filterBounds(shape)

lithology_c = lithology_c.set("system:time_start", ee.Date("2022-01-01"))
arr_lithology = lithology_c.wx.to_xarray(scale=10000, crs='EPSG:4326')

xds_repr_match_lithology = arr_lithology.rio.reproject_match(soil_wc, resampling = rasterio.enums.Resampling.mode)

xds_repr_match_lithology


# get class lithology file

In [ ]:
df_1 = pd.read_excel("/content/drive/MyDrive/w/Lithology_table.xlsx", sheet_name="Sheet1")
df_1

In [ ]:
column_titles = df_1.columns.tolist()
omitted_list = column_titles[1:]
column_titles

# map dataframe on lithology data

In [ ]:
column_titles = df_1.columns.tolist()
omitted_list = column_titles[1:]

df = xds_repr_match_lithology.to_dataframe()

for col in omitted_list:

    land_use_fao_dict = dict(zip(df_1['Value'], df_1[col]))

    df[col] = df['b1'].map(land_use_fao_dict)


lithology_name_da = df.to_xarray()
lithology_name_da = lithology_name_da.transpose('time', 'y', 'x')
lithology_name_da

In [ ]:
xds_repr_match_lithology.to_netcdf("/content/drive/MyDrive/w/lithology_2022001.nc")